In [1]:
# Imports
import numpy as np
from sklearn import preprocessing


# Preprocessing


In [2]:
observedVariable = preprocessing.LabelEncoder();
observedVariable.fit(['b','g','y','r']);

latentVariable = preprocessing.LabelEncoder();
latentVariable.fit(['1:1','1:2','1:3','1:4','2:1','2:2','2:3','2:4','3:1','3:2','3:3','3:4','4:1','4:2','4:3','4:4']);


# Training

In [10]:
transitionProbabilities = np.zeros((16,16));
emissionProbabilities = np.zeros((16,4));
priorProbablities = np.zeros((16));
    
#Get observation counts
with open('../robot_train.data', 'r') as file:
    flag = 0;
    for line in file:
        if (line != '.\n'):
            split = line.strip('\n').split(" ")
            currentLatentState  =  latentVariable.transform(split[0])
            emissionProbabilities[currentLatentState,observedVariable.transform(split[1])] += 1
                
            if (flag == 1):
                transitionProbabilities[currentLatentState, prevLatentState] += 1
                transitionProbabilities[prevLatentState, currentLatentState] += 1
                    
            prevLatentState = currentLatentState;
            flag = 1;
        
        else:
            priorProbablities[latentVariable.transform(split[0])] += 1
            flag = 0;

# Normalize counts
priorProbablities /= np.sum(priorProbablities)
    
for i in range(transitionProbabilities.shape[1]):
    sum = np.sum(transitionProbabilities[:,i])
    if (sum != 0):
        transitionProbabilities[:,i] /= sum

for i in range(emissionProbabilities.shape[0]):
    sum = np.sum(emissionProbabilities[i,:])
    if (sum != 0):
        emissionProbabilities[i,:] /= sum



# Viterbi

In [18]:
def Viterbi(sequence):
    
        m = transitionProbabilities.shape[0]
        n = sequence.shape[0]
    
        P = np.zeros((m,n))
        S = np.zeros((m,n))
        x = np.zeros((n))
    
        P[:,0] = priorProbablities * emissionProbabilities[:,sequence[0]]
        
        for i in range(1,n):
            P[:,i] = np.max(P[:,i-1, None].dot(emissionProbabilities[:,sequence[i],None].T)*transitionProbabilities, axis=0)
            S[:,i] = np.argmax(np.tile(P[:,i-1, None],m)*transitionProbabilities, axis=0)

        x = [P[:,-1].argmax()]
        for i in range(n-1, 0, -1):
            x.append(S[x[-1],i])

        return np.array(x[::-1])


# Testing

In [19]:
average=0;
count=0;
with open('../robot_test.data', 'r') as file:
    sequence = []
    labels   = []
    for line in file:
        if (line != '.\n'):
            temp = line.strip('\n').split(" ")
            sequence.append(temp[1])
            labels.append(temp[0])
        else:
            sequence = observedVariable.transform(sequence)
            labels = latentVariable.transform(labels)
            predicted = Viterbi(sequence)
            accuracy = np.average(labels == predicted) * 100;            
            average = average + accuracy;
            count = count + 1;           
            sequence = []
            labels = []
            
print average/count;



81.29


/home/siddharth/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:18: DeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
